In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("data/train.csv")
X = df_train.iloc[:, :-1].values
y = df_train.iloc[:, -1].values

df_kaggle = pd.read_csv("data/test.csv")
X_kaggle = df_kaggle.iloc[:, :].values

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder

# ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(handle_unknown="ignore"), [0, 1])], remainder="passthrough")
# X = np.array(ct.fit_transform(X))
# X_kaggle = np.array(ct.transform(X_kaggle))

In [3]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X
y_train = y

In [ ]:
y_train

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train[:, 84:89] = sc.fit_transform(X_train[:, 84:89])
# X_test[:, 33:] = sc.transform(X_test[:, 33:])
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1))

sc_kaggle = StandardScaler()
X_kaggle[:, 84:89] = sc_kaggle.fit_transform(X_kaggle[:, 84:89])

In [5]:
from catboost import CatBoostRegressor

regressor = CatBoostRegressor()
regressor.fit(X_train, y_train)

Learning rate set to 0.051375
0:	learn: 0.9899583	total: 137ms	remaining: 2m 16s
1:	learn: 0.9801019	total: 138ms	remaining: 1m 8s
2:	learn: 0.9704306	total: 139ms	remaining: 46.3s
3:	learn: 0.9608837	total: 141ms	remaining: 35s
4:	learn: 0.9515362	total: 142ms	remaining: 28.3s
5:	learn: 0.9423312	total: 144ms	remaining: 23.8s
6:	learn: 0.9332527	total: 145ms	remaining: 20.6s
7:	learn: 0.9242867	total: 146ms	remaining: 18.1s
8:	learn: 0.9155277	total: 148ms	remaining: 16.3s
9:	learn: 0.9069430	total: 149ms	remaining: 14.8s
10:	learn: 0.8984110	total: 151ms	remaining: 13.5s
11:	learn: 0.8901173	total: 152ms	remaining: 12.5s
12:	learn: 0.8819625	total: 153ms	remaining: 11.6s
13:	learn: 0.8739207	total: 154ms	remaining: 10.9s
14:	learn: 0.8660251	total: 155ms	remaining: 10.2s
15:	learn: 0.8582804	total: 157ms	remaining: 9.65s
16:	learn: 0.8506699	total: 159ms	remaining: 9.16s
17:	learn: 0.8431389	total: 160ms	remaining: 8.74s
18:	learn: 0.8357888	total: 162ms	remaining: 8.34s
19:	learn: 0

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix

# y_pred = classifier.predict(X_test)
# acc_sc = accuracy_score(y_test, y_pred)
# cf_mx = confusion_matrix(y_test, y_pred)
# print("Accuracy Score:", acc_sc)
# print(cf_mx)

In [6]:
from sklearn.model_selection import cross_val_score
from datetime import datetime as dt
timestamp = dt.today()

accuracies = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10)
print(f"Accuracy: {accuracies.mean()}")
print(f"StdDev: {accuracies.std()}")
try:
    with open("data/cross_val.log", mode="x+") as cvlog:
        cvlog.write("timestamp,accuracy,stddev")
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")
except:
    with open("data/cross_val.log", mode="a") as cvlog:
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")

Learning rate set to 0.050419
0:	learn: 0.5035803	total: 7.55ms	remaining: 7.54s
1:	learn: 0.4975239	total: 9.28ms	remaining: 4.63s
2:	learn: 0.4913264	total: 10.7ms	remaining: 3.57s
3:	learn: 0.4854143	total: 12.1ms	remaining: 3.02s
4:	learn: 0.4795989	total: 13.5ms	remaining: 2.69s
5:	learn: 0.4738618	total: 14.8ms	remaining: 2.45s
6:	learn: 0.4681881	total: 16.1ms	remaining: 2.28s
7:	learn: 0.4625950	total: 17.4ms	remaining: 2.15s
8:	learn: 0.4569500	total: 18.7ms	remaining: 2.06s
9:	learn: 0.4515103	total: 20.1ms	remaining: 1.99s
10:	learn: 0.4460186	total: 21.4ms	remaining: 1.93s
11:	learn: 0.4407250	total: 22.5ms	remaining: 1.85s
12:	learn: 0.4355000	total: 23.9ms	remaining: 1.81s
13:	learn: 0.4301438	total: 25.2ms	remaining: 1.77s
14:	learn: 0.4249546	total: 26.5ms	remaining: 1.74s
15:	learn: 0.4199360	total: 27.8ms	remaining: 1.71s
16:	learn: 0.4149847	total: 29.1ms	remaining: 1.68s
17:	learn: 0.4100993	total: 30.4ms	remaining: 1.66s
18:	learn: 0.4052783	total: 31.7ms	remaining

In [10]:
y_kaggle = sc_y.inverse_transform(regressor.predict(X_kaggle))

In [11]:
with open(f"data/kaggle/kaggle_{timestamp.strftime('%d-%m-%Y_%Hh%M')}.csv", mode="w") as out:
    out.write("Id,preco")
    for i in range(len(y_kaggle)):
        out.write(f"\n{i},{round(y_kaggle[i], 2):.2f}")